<a href="https://colab.research.google.com/github/huavanson/CS114.K21/blob/master/sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mô tả bài toán : Phân biệt 1 tiêu đề có tính châm biếm hay không 


*   input : 1 title 
*   output : is sarcasm or not sarcasm 








Dataset : lấy từ kaggle (link dataset:https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection)
Mô tả dataset : có 3 columns

*   headline : là tiêu đề của 1 bài viết 
*   is_sarcastic: 0 là not sarcasm , 1 là is sarcasm


*   article_link: link bài viết 





Dataset lấy trên kaggle có 2 file nhưng em chỉ sử dụng 1 file 

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sonhuavan","key":"899cfa81c0f3f4051aee96eb7e74b10f"}'}

Sau khi up file kaggle.json lên thì tạo 1 thư mục có tên .kaggle đưa file vừa up vào 

In [ ]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 65 Jun 22 08:22 kaggle.json


Bước này Download dataset trên kaggle về 


In [ ]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 111MB/s]


Check xem các thư viện cần thiết đã được install vào chưa 


In [ ]:
!pip install numpy
!pip install pandas 
!pip install sklearn

In [ ]:
import json
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


Đọc dataset bằng Pandas và check số lượng data 

In [ ]:
df=pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)
# shape and head
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
df.shape

(26709, 3)

check xem data có bị missing value hay không

In [ ]:
print(df.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


Column article_link không cần thiết để detect 1 headline có sarcasm hay không nên có thể được lượt bỏ 


In [ ]:
df = df.drop(['article_link'], axis=1)

Dữ liệu không bị missing value nhưng cột tiêu đề nên được loại bỏ các kí tự không cần thiết vì vậy em sử dụng Regular Expression ( Biểu thức chính quy) để loại bỏ các kí tự đặc biệt đó ví dụ dấu nháy dấu chấm v.v

In [ ]:
df['headline'] = df['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))


In [ ]:
df.head(5)

,headline,is_sarcastic
0,former versace store clerk sues over secret b...,0
1,the roseanne revival catches up to our thorn...,0
2,mom starting to fear son s web series closest ...,1
3,boehner just wants wife to listen not come up...,1
4,j k rowling wishes snape happy birthday in th...,0


In [ ]:
features = df['headline']
labels = df['is_sarcastic']
features_compare=features


Xử dụng PorterStemmer để cắt giảm 1 từ thành từ gốc của nó 
Ví dụ : "reading" or "reader"
Sau khi stemming sẽ trở thành "read" ( cho cả 2 từ trên)

In [ ]:
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

So sánh chuỗi trước và sau khi stemming 

In [ ]:
print(features_compare[0])
print(features[0])

former versace store clerk sues over secret  black code  for minority shoppers
former versac store clerk sue over secret black code for minor shopper


tf-idf ( term frequency–inverse document frequency) hay còn gọi là tần số xuất hiện- tần số nghịch đảo của 1 từ trong 1 văn bản . Kỹ thuật này sử dụng để đếm số lần xuất hiện của 1 từ trong 1 văn bản nhiều hoặc ít để tìm ra những từ quan trọng trong 1 văn bản 

Link tham khảo tf-idf :https://en.wikipedia.org/wiki/Tf%E2%80%93idf

TfidfVectorizer dùng để chuyển đổi bộ dữ liệu sang 1 ma trận có các thông số tf-idf dưới đây là link thư viện scikitlearn


https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features).toarray()

features sau khi được chuyển sang ma trận 


In [ ]:
features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
print(features.shape)

(26709, 5000)


Vì dữ liệu khá nhiều nên em sẽ chia theo tỷ lệ 90-10

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.1, random_state = 0)

Dùng thử 4 model để train


---



---



In [ ]:
# model 1:-
# linear support vector classifier
lsvc = LinearSVC()
# training model
lsvc.fit(features_train, labels_train)
# accuracy tập train và tập test
print(lsvc.score(features_train, labels_train))
print(lsvc.score(features_test, labels_test))   
# model 2:-
#  Gaussuan Naive Bayes
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print(gnb.score(features_train, labels_train))
print(gnb.score(features_test, labels_test))  
# model 3:-
# Logistic Regression
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print(lr.score(features_train, labels_train)) 
print(lr.score(features_test, labels_test))     
# model 4:
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print(rfc.score(features_train, labels_train))
print(rfc.score(features_test, labels_test))    


0.9110574923038522
0.8360164732309996
0.7826358266078709
0.7162111568700861
0.8824777435726766
0.8375140396855111
0.9895998003161661
0.7963309621864471


Crawl Data trên trang https://www.theonion.com/latest để lấy những head line sarcasm

Install 1 số thư viện cần thiết

In [ ]:
!pip install beautifulsoup4
! pip install requests

Hàm Crawl data sử dụng beautifulSoup 

BeautifulSoup(BS) là 1 package của python dùng để cạo thông tin từ 1 trang web . BS cho phép phân tích nội dung HTML của 1 URL nhất định và truy cập các phần tử của nó bằng cách xác định các thẻ và các thuộc tính . 

Link tham khảo cách cào dữ liệu trên 1 trang web :https://towardsdatascience.com/web-scraping-news-articles-in-python-9dd605799558

In [ ]:
import requests
from bs4 import BeautifulSoup
def crawl(url, data):
  r1 = requests.get(url)
  coverpage = r1.content
  soup1 = BeautifulSoup(coverpage, 'html5lib')
  coverpage_news = soup1.find_all("h2", class_="sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE")
  for i in coverpage_news:
     data.append(i.get_text())

để cung cấp BeautifulSoup mã HTML của bất kỳ trang nào,  sẽ cần import 1 cái module requests

Vì em cần cạo khoảng 1000 headline nhưng mỗi trang trên trang web :"https://www.theonion.com/latest" chỉ hiển thị khoảng 20 headline, trang kế tiếp có URL khác , để chuyển sang trang tiếp theo em trích xuất vào chổ chuyển trang cụ thể là nút(more stories trong trang web) để tìm ra đuôi href để gắn vào link đầu tiên để truy cập vào trang kế tiếp cào tiếp 20 headline, như vậy dữ liệu cào xuống sẽ không trùng nhau 




In [ ]:
url = "https://www.theonion.com/latest"
more_stories = ''
data= []
while len(data)<1000:
  crawl(url, data)
  r1 = requests.get(url)
  coverpage = r1.content
  soup1 = BeautifulSoup(coverpage, 'html5lib')
  more_stories = soup1.find_all("a", class_="sc-1out364-0 hMndXN js_link")
  more_stories_next_link = more_stories[-1].get('href')
  url="https://www.theonion.com/latest"+more_stories_next_link
  print(url)

https://www.theonion.com/latest?startTime=1592484780423
https://www.theonion.com/latest?startTime=1592316960776
https://www.theonion.com/latest?startTime=1591985280204
https://www.theonion.com/latest?startTime=1591811340142
https://www.theonion.com/latest?startTime=1591712700562
https://www.theonion.com/latest?startTime=1591379640976
https://www.theonion.com/latest?startTime=1591203180902
https://www.theonion.com/latest?startTime=1591048080455
https://www.theonion.com/latest?startTime=1590757260552
https://www.theonion.com/latest?startTime=1590598140377
https://www.theonion.com/latest?startTime=1590408300419
https://www.theonion.com/latest?startTime=1590090960434
https://www.theonion.com/latest?startTime=1589985540710
https://www.theonion.com/latest?startTime=1589828820662
https://www.theonion.com/latest?startTime=1589546580875
https://www.theonion.com/latest?startTime=1589382780197
https://www.theonion.com/latest?startTime=1589287860502
https://www.theonion.com/latest?startTime=158896

độ dài data sau khi cào

In [ ]:
data=list(dict.fromkeys(data))
print(len(data))

1002


Biểu diễn data dưới dạng bảng

In [ ]:
is_sarcastic=[]
for i in range(0,len(data)):
   is_sarcastic.append('1')
test_data_1=pd.DataFrame([data,is_sarcastic]).T
test_data_1.columns=['headline','is_sarcastic']  
test_data_1.head()

,headline,is_sarcastic
0,Mental Health Experts Advise On Best Ways To C...,1
1,The Onion’s Father’s Day Gift Guide For The Da...,1
2,Staples Center Employee Realizes He Left Shot ...,1
3,Father’s Day Gifts That Will Make Your Dad Fee...,1
4,"Huh, Boyfriend’s Ex Just Made Interesting Hair...",1


In [ ]:
test_data_1.shape

(1002, 2)